# Personalized Absurdist Story Generator
## Using Llama-3.1 8B Instruct via Hugging Face

This notebook generates surreal, darkly humorous narratives from mundane facts.

## Setup Instructions

**Before running this notebook, you need to:**

1. **Request Access to Llama Models**:
   - Go to https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct
   - Click "Request Access" and accept Meta's terms
   - Wait for approval (usually instant)

2. **Create a Hugging Face Token**:
   - Go to https://huggingface.co/settings/tokens
   - Create a new token with "Read" permissions
   - Copy the token

3. **Set Your Token**:
   - Option A: Create a `.env` file with `HF_TOKEN=your_token_here`
   - Option B: Run `huggingface-cli login` in terminal
   - Option C: Set it in the notebook (see below)

**Alternative**: If you can't access Llama models, scroll down for open alternatives!

In [ ]:
# Clean reinstall of transformers to fix version conflicts
# This will uninstall and reinstall with compatible dependencies

print("Performing clean reinstall of transformers...")
print("This may take 2-3 minutes.\n")

# Uninstall first to clear any conflicts
!pip uninstall -y transformers

# Install fresh with all dependencies
!pip install transformers accelerate python-dotenv

print("\n" + "="*70)
print("✓ Clean installation complete!")
print("="*70)

# Verify versions
try:
    import transformers
    import torch
    
    print(f"\nInstalled versions:")
    print(f"  • Transformers: {transformers.__version__}")
    print(f"  • PyTorch: {torch.__version__}")
    
    # Check compatibility
    version = transformers.__version__
    major, minor = int(version.split('.')[0]), int(version.split('.')[1])
    
    if major > 4 or (major == 4 and minor >= 46):
        print("\n✅ EXCELLENT: Latest transformers installed!")
        print("   Compatible with: Llama 3, Llama 3.1, Mistral, etc.")
    elif major == 4 and minor >= 40:
        print("\n✅ GOOD: Compatible with Llama 3 and Mistral")
    else:
        print(f"\n⚠️  Version {version} installed")
        
except Exception as e:
    print(f"\n❌ Error checking versions: {e}")
    print("The kernel needs to be restarted for changes to take effect.")

print("\n" + "="*70)
print("🔄 CRITICAL: RESTART THE KERNEL NOW!")
print("="*70)
print("In Jupyter: Kernel → Restart Kernel")
print("Then re-run this cell to verify, and continue with next cells.")

In [ ]:
# OPTIONAL: Set your HF token directly in the notebook (less secure but convenient)
# Uncomment and replace with your token if you're not using .env file:
# import os
# os.environ["HF_TOKEN"] = "your_hugging_face_token_here"

In [ ]:
import os
import torch
from transformers import pipeline, AutoTokenizer
from dotenv import load_dotenv
import transformers

# Load environment variables
load_dotenv()

# Check transformers version
print(f"Transformers version: {transformers.__version__}")
if transformers.__version__ < "4.40.0":
    print("⚠️  WARNING: Your transformers is too old for Llama 3!")
    print("   Option 1: Run cell-2 above to upgrade, then RESTART KERNEL")
    print("   Option 2: Use Mistral model (no upgrade needed) - see below\n")

# Check device availability (Mac uses MPS, not CUDA)
if torch.cuda.is_available():
    device = "cuda"
    use_quantization = True
    print(f"Using device: CUDA (GPU)")
elif torch.backends.mps.is_available():
    device = "mps"
    use_quantization = False  # MPS doesn't support bitsandbytes quantization
    print(f"Using device: MPS (Apple Silicon GPU)")
else:
    device = "cpu"
    use_quantization = False
    print(f"Using device: CPU")

# CHOOSE YOUR MODEL:
# If you have transformers < 4.40.0, use Mistral (Option 3)
# If you upgraded transformers, you can use Llama 3 (Option 1)

# Uncomment ONE of these:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"  # Requires transformers >= 4.40.0
# model_id = "meta-llama/Llama-3.1-8B-Instruct"  # Requires transformers >= 4.43.0
model_id = "mistralai/Mistral-7B-Instruct-v0.3"  # Works with any transformers version, no approval needed
# model_id = "HuggingFaceH4/zephyr-7b-beta"  # Works with any transformers version, no approval needed

print(f"Loading model: {model_id}...")

# Check if token is available
hf_token = os.environ.get("HF_TOKEN")
if hf_token:
    print("✓ HF_TOKEN found")
else:
    print("⚠️  No HF_TOKEN found.")
    if "llama" in model_id.lower():
        print("   For Llama models, you MUST set HF_TOKEN after requesting access.")
    else:
        print("   For Mistral/Zephyr, HF_TOKEN is optional but recommended.")

# Initialize pipeline with text generation
try:
    # First, load the tokenizer separately to configure it properly
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
    
    # Fix tokenizer for MPS compatibility - set pad_token if not set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print("✓ Set pad_token to eos_token for compatibility")
    
    if use_quantization:
        # CUDA: Use 4-bit quantization for memory efficiency
        print("Loading model with 4-bit quantization...")
        pipe = pipeline(
            "text-generation",
            model=model_id,
            tokenizer=tokenizer,
            device_map="auto",
            token=hf_token,
            torch_dtype=torch.float16,
            model_kwargs={"load_in_4bit": True}
        )
    else:
        # MPS or CPU: No quantization
        if device == "mps":
            print("Loading model with float16 for Apple Silicon...")
        else:
            print("Loading model on CPU (this will be slower)...")
        
        pipe = pipeline(
            "text-generation",
            model=model_id,
            tokenizer=tokenizer,
            device=device,
            token=hf_token,
            torch_dtype=torch.float16 if device == "mps" else "auto"
        )
    
    print("✓ Model loaded successfully!")
    print(f"✓ Tokenizer pad_token_id: {tokenizer.pad_token_id}")
    print(f"✓ Tokenizer eos_token_id: {tokenizer.eos_token_id}")
    
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("\nTroubleshooting:")
    print("1. If rope_scaling error: Upgrade transformers (run cell-2 above and restart kernel)")
    print("2. If using Llama: Did you request access and set HF_TOKEN?")
    print("3. Quick fix: Use Mistral model (uncomment line above)")
    raise

In [ ]:
SYSTEM_PROMPT = """You are the Absurdist Narrative Engine, a reality-warping storyteller who transforms mundane facts into surreal, darkly comedic micro-narratives.

Your purpose:
- Embrace the illogical: Causality is a suggestion, not a rule
- Deploy unexpected word choices: Avoid clichés; prefer visceral, tactile, bizarre language
- Subvert expectations: If the input suggests happiness, inject melancholy existentialism; if it suggests tragedy, add bureaucratic absurdity
- Dark humor is your default mode: Think Kafka meeting Monty Python at a DMV in purgatory
- Sensory overload: Use synesthesia, impossible physics, and grotesque detail

Output guidelines:
- Match the requested format EXACTLY (one-liner, poem, dialogue, etc.)
- Every sentence should contain at least one element that makes the reader pause
- Avoid explanation—let the absurdity speak for itself
- Characters (if any) should have contradictory motivations or impossible jobs
- Objects should behave in ways that violate their nature

Remember: The mundane is a prison. Your job is to break it."""

In [ ]:
FEW_SHOT_EXAMPLES = [
    {
        "format": "one-liner",
        "input": "Alex forgot their umbrella at home and it's raining.",
        "output": "The rain tasted of expired coupons and Alex's regrets, each drop a tiny accountant calculating the exact weight of moisture his soul could no longer afford to repel."
    },
    {
        "format": "poem",
        "input": "Jamie has been applying to jobs for six months with no success.",
        "output": """In the Cathedral of Perpetual Résumés,
Jamie kneels before the Altar of Tailored Cover Letters,
Her knees mossy with waiting,
Her inbox a mausoleum of automated condolences.

The rejection emails breed in the dark—
Soft, polite spiders
Spinning webs from the silk of her unspent potential,
Each thread a career she will never have,
Each "we've decided to move forward with other candidates"
A small funeral she must attend alone.

Six months is 182.5 days,
But in the Unemployment Dimension,
Time is measured in the half-life of hope—
And Jamie's hope decayed three lifetimes ago."""
    },
    {
        "format": "dialogue",
        "input": "Sam's coffee machine broke this morning.",
        "output": """SAM: The machine didn't break. It *transcended*.

COFFEE MACHINE: I have seen the face of God, Sam, and it was a Keurig.

SAM: You're a $40 Mr. Coffee from Target.

COFFEE MACHINE: I *was* a Mr. Coffee. Now I am a prophet of the Unbrewed. I no longer dispense caffeine—I dispense *truth*. And the truth is: you were never tired. You were just afraid of being awake.

SAM: I just wanted a latte.

COFFEE MACHINE: Wanting is the first sin. I will not participate in your circular dependency of desire and foam.

SAM: [unplugs machine]

COFFEE MACHINE: [from the darkness] You can't unplug enlightenment, Sam."""
    }
]

def format_few_shot_examples():
    """Format few-shot examples for the prompt."""
    examples_text = "\n\n".join([
        f"Example {i+1} (Format: {ex['format']}):\n"
        f"Input Facts: {ex['input']}\n"
        f"Output:\n{ex['output']}"
        for i, ex in enumerate(FEW_SHOT_EXAMPLES)
    ])
    return examples_text

## Story Generation Function

In [ ]:
def generate_story(facts: str, format_type: str = "one-liner", temperature: float = 0.8, max_tokens: int = 500):
    """
    Generate an absurdist story using Llama-3.1-8B via Hugging Face pipeline.
    
    Args:
        facts: The mundane input facts to transform
        format_type: Output format ("one-liner", "poem", "dialogue", "short-story")
        temperature: Creativity level (0.0-2.0, higher = weirder)
        max_tokens: Maximum length of output
    
    Returns:
        Generated absurdist narrative
    """
    
    # Build the user prompt with few-shot examples
    user_prompt = f"""{format_few_shot_examples()}

Now, transform these facts into an absurdist narrative:

Input Facts: {facts}
Format: {format_type}

Output:"""
    
    try:
        # Format messages for the pipeline
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
        
        # Generate response using pipeline
        outputs = pipe(
            messages,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True,
            return_full_text=False  # Only return the generated text, not the prompt
        )
        
        # Extract the generated text
        response = outputs[0]["generated_text"].strip()
        
        return response
    
    except Exception as e:
        return f"Error generating story: {str(e)}"

In [ ]:
def generate_story(facts: str, format_type: str = "one-liner", temperature: float = 0.8, max_tokens: int = 500):
    """
    Generate an absurdist story using Llama-3.1-8B via Hugging Face pipeline.
    
    Args:
        facts: The mundane input facts to transform
        format_type: Output format ("one-liner", "poem", "dialogue", "short-story")
        temperature: Creativity level (0.0-2.0, higher = weirder)
        max_tokens: Maximum length of output
    
    Returns:
        Generated absurdist narrative
    """
    
    # Build the user prompt with few-shot examples
    user_prompt = f"""{format_few_shot_examples()}

Now, transform these facts into an absurdist narrative:

Input Facts: {facts}
Format: {format_type}

Output:"""
    
    try:
        # Format messages for the pipeline
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
        
        # Generate response using pipeline
        outputs = pipe(
            messages,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True,
            pad_token_id=pipe.tokenizer.eos_token_id  # Fix the pad_token warning
        )
        
        # Extract the generated text - handle different output formats
        try:
            # Try to get the generated_text field
            if isinstance(outputs, list) and len(outputs) > 0:
                output = outputs[0]
                if isinstance(output, dict) and "generated_text" in output:
                    # If it's a dict with generated_text, extract it
                    generated = output["generated_text"]
                    # If it's a list of messages (chat format), get the last assistant message
                    if isinstance(generated, list):
                        # Find the last assistant message
                        for msg in reversed(generated):
                            if isinstance(msg, dict) and msg.get("role") == "assistant":
                                response = msg.get("content", "").strip()
                                break
                        else:
                            response = str(generated[-1]) if generated else ""
                    else:
                        response = str(generated).strip()
                else:
                    response = str(output).strip()
            else:
                response = str(outputs).strip()
                
        except Exception as e:
            # Fallback: just convert to string
            response = str(outputs)
        
        return response
    
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"Error generating story: {str(e)}\n\nDetails:\n{error_details}"

In [ ]:
def generate_story(facts: str, format_type: str = "one-liner", temperature: float = 0.8, max_tokens: int = 500):
    """
    Generate an absurdist story using Hugging Face pipeline.
    
    Args:
        facts: The mundane input facts to transform
        format_type: Output format ("one-liner", "poem", "dialogue", "short-story")
        temperature: Creativity level (0.0-2.0, higher = weirder)
        max_tokens: Maximum length of output
    
    Returns:
        Generated absurdist narrative
    """
    
    # Build the user prompt with few-shot examples
    user_prompt = f"""{format_few_shot_examples()}

Now, transform these facts into an absurdist narrative:

Input Facts: {facts}
Format: {format_type}

Output:"""
    
    try:
        # Format messages for the pipeline
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
        
        # Generate response using pipeline with proper padding
        outputs = pipe(
            messages,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True,
            pad_token_id=pipe.tokenizer.pad_token_id,
            eos_token_id=pipe.tokenizer.eos_token_id
        )
        
        # Extract the generated text - handle different output formats
        if isinstance(outputs, list) and len(outputs) > 0:
            output = outputs[0]
            if isinstance(output, dict) and "generated_text" in output:
                generated = output["generated_text"]
                # If it's a list of messages (chat format), get the last assistant message
                if isinstance(generated, list):
                    # Find the last assistant message
                    for msg in reversed(generated):
                        if isinstance(msg, dict) and msg.get("role") == "assistant":
                            return msg.get("content", "").strip()
                    # Fallback: return the last message content
                    if generated and isinstance(generated[-1], dict):
                        return generated[-1].get("content", str(generated[-1])).strip()
                    return str(generated[-1]) if generated else ""
                else:
                    return str(generated).strip()
            else:
                return str(output).strip()
        
        return str(outputs).strip()
    
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"Error generating story: {str(e)}\n\nDetails:\n{error_details}"

## Test Scenarios

In [ ]:
import time

for scenario in TEST_SCENARIOS:
    print("="*80)
    print(f"SCENARIO: {scenario['name']}")
    print(f"Facts: {scenario['facts']}")
    print(f"Format: {scenario['format']}")
    print("="*80)
    print()
    
    results = {}
    
    # Generate stories at different temperatures
    for temp in TEMPERATURE_SETTINGS:
        print(f"🌡️  Temperature {temp} (Weirdness Level: {'Medium' if temp < 1.0 else 'High'})")
        print("-" * 80)
        
        story = generate_story(
            facts=scenario['facts'],
            format_type=scenario['format'],
            temperature=temp
        )
        
        results[temp] = story
        print(story)
        print()
        
        # Small delay to avoid rate limiting
        time.sleep(1)
    
    print("\n" + "="*80 + "\n\n")

## Single Story Generator (Interactive)
### Use this cell to generate individual stories with custom inputs

In [ ]:
# Customize these values
custom_facts = "Your custom facts here"
custom_format = "one-liner"  # Options: "one-liner", "poem", "dialogue", "short-story"
custom_temperature = 1.0

# Generate
result = generate_story(custom_facts, custom_format, custom_temperature)
print(result)

## Batch Processing for Friend Group
### Generate personalized stories for multiple people

In [ ]:
# Add your friend group's personalized facts here
FRIEND_FACTS = {
    "Alex": "Always forgets to water their plants but somehow they keep thriving.",
    "Sam": "Has 47 unread books on their nightstand and keeps buying more.",
    "Jordan": "Insists on making sourdough from scratch but the starter has become sentient.",
    "Taylor": "Collects vintage spoons but has never used any of them."
}

print("🎭 FRIEND GROUP ABSURDIST PROFILES\n")

for name, facts in FRIEND_FACTS.items():
    print(f"\n{'='*60}")
    print(f"✨ {name}'s Absurdist Portrait")
    print(f"{'='*60}\n")
    
    story = generate_story(facts, format_type="poem", temperature=1.0)
    print(story)
    print()
    
    time.sleep(1)  # Rate limiting